In [17]:
# Importing the necessary libraries
import pandas as pd  # For data manipulation and analysis
from sklearn.model_selection import train_test_split  # To split data into training and testing sets
from sklearn.preprocessing import MinMaxScaler  # To normalize data
from sklearn.feature_selection import RFE  # Recursive Feature Elimination (RFE) for feature selection
from sklearn.linear_model import LogisticRegression  # Logistic Regression model
from sklearn.tree import DecisionTreeClassifier  # Decision Tree model
from sklearn.ensemble import RandomForestClassifier  # Random Forest model
from sklearn.svm import SVC  # Support Vector Machine (SVM) model
from sklearn.neighbors import KNeighborsClassifier  # K-Nearest Neighbors (KNN) model
from sklearn.metrics import accuracy_score  # To calculate prediction accuracy

# Imports for building the neural network model with Keras
from tensorflow.keras.models import Sequential  # To create sequential models
from tensorflow.keras.layers import Dense, Dropout  # Fully connected (Dense) layers and Dropout for regularization
from tensorflow.keras.optimizers import Adam  # Adam optimizer for network training
import warnings
warnings.filterwarnings("ignore")

In [18]:
# --------------------------------------------------------------------
# 1. Dataset Loading
# --------------------------------------------------------------------
# Loads the churn dataset from a CSV file. 
# This dataset contains customer information and whether they churned or not.
df = pd.read_csv(r'C:\Users\julia\Repos\juliano\TensorFlow\Churn.csv')

In [19]:
# --------------------------------------------------------------------
# 2. Data Preprocessing
# --------------------------------------------------------------------
# Processing the 'Total Charges' column:
# - Replaces empty values ('') with 0
df['Total Charges'] = df['Total Charges'].replace('', 0)

# - Fills null values (NaN) with 0
df['Total Charges'] = df['Total Charges'].fillna(0)

# - Converts the 'Total Charges' column to numeric (float), handling errors and replacing possible NaN with 0
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce').fillna(0)

In [20]:
# --------------------------------------------------------------------
# 3. Normalization of Numerical Data
# --------------------------------------------------------------------
# Creates a MinMaxScaler object to normalize data between 0 and 1
scaler = MinMaxScaler()

# Normalizes the 'tenure', 'Monthly Charges', and 'Total Charges' columns
df[['tenure', 'Monthly Charges', 'Total Charges']] = scaler.fit_transform(
    df[['tenure', 'Monthly Charges', 'Total Charges']]
)

In [21]:
# --------------------------------------------------------------------
# 4. Data Preparation for Modeling
# --------------------------------------------------------------------
# Separates features (explanatory variables) and the target variable
# - Removes the 'Churn' column (which will be the target variable) and 'Customer ID' (not useful for prediction)
# - Converts categorical variables into dummy variables (one-hot encoding)
X = pd.get_dummies(df.drop(['Churn', 'Customer ID'], axis=1))

# Creates the target variable 'y', transforming 'Yes' into 1 and 'No' into 0
y = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

In [22]:
# --------------------------------------------------------------------
# 5. Splitting Data into Training and Testing Sets
# --------------------------------------------------------------------
# Splits data into 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# --------------------------------------------------------------------
# 6. Feature Selection Using RFE (Recursive Feature Elimination)
# --------------------------------------------------------------------
# Uses logistic regression as the base estimator for RFE
model_lr = LogisticRegression()

# Configures RFE to select the top 10 features based on their importance determined by the model
selector = RFE(model_lr, n_features_to_select=10)

# Fits (trains) RFE with the training data
selector = selector.fit(X_train, y_train)

# Identifies the column names selected by RFE
selected_columns = X_train.columns[selector.support_]
print("Selected columns:", selected_columns)

# Filters datasets to keep only the selected features
X_train_selected = X_train[selected_columns]
X_test_selected = X_test[selected_columns]

Selected columns: Index(['tenure', 'Total Charges', 'Multiple Lines_No',
       'Internet Service_Fiber optic', 'Online Security_No', 'Tech Support_No',
       'Contract_Month-to-month', 'Contract_Two year', 'Paperless Billing_No',
       'Payment Method_Electronic check'],
      dtype='object')


In [24]:
# --------------------------------------------------------------------
# 7. Function for Training and Evaluating Classic Models
# --------------------------------------------------------------------
def train_and_evaluate(model, model_name):
    """
    Trains a machine learning model, makes predictions on the test set, 
    and prints the model's accuracy.
    
    Parameters:
        model : the model object to be trained (e.g., LogisticRegression, DecisionTreeClassifier, etc.)
        model_name : name of the model (string) for displaying results
    """
    # Trains the model using the training data with selected features
    model.fit(X_train_selected, y_train)
    
    # Makes predictions on the test set
    y_hat = model.predict(X_test_selected)
    
    # Calculates accuracy by comparing predictions with actual test values
    accuracy = accuracy_score(y_test, y_hat)
    
    # Prints accuracy formatted to 4 decimal places
    print(f"Accuracy of the {model_name} model: {accuracy:.4f}")
    
    return accuracy

In [25]:
# --------------------------------------------------------------------
# 8. Training and Evaluating Classic Machine Learning Models
# --------------------------------------------------------------------
# Creates a dictionary containing the models to be tested
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier()
}

# Iterates over each model, trains and evaluates, displaying the accuracy of each
for name, model in models.items():
    train_and_evaluate(model, name)


Accuracy of the Logistic Regression model: 0.8013
Accuracy of the Decision Tree model: 0.7410
Accuracy of the Random Forest model: 0.7800
Accuracy of the SVM model: 0.7935
Accuracy of the KNN model: 0.7850


In [26]:
# --------------------------------------------------------------------
# 9. Building, Training, and Evaluating a Neural Network Model with Keras
# --------------------------------------------------------------------
# Creates a sequential model (linear stacking of layers)
model_nn = Sequential()

# Adds the first hidden layer:
# - 64 neurons
# - Activation function: 'relu'
# - 'input_dim' set to the number of selected features
model_nn.add(Dense(units=64, activation='relu', input_dim=len(X_train_selected.columns)))

# Adds a Dropout layer to reduce overfitting, randomly discarding 50% of neurons during training
model_nn.add(Dropout(0.5))

# Adds a second hidden layer with 128 neurons and 'relu' activation function
model_nn.add(Dense(units=128, activation='relu'))

# Adds the output layer:
# - 1 neuron, as this is a binary classification problem
# - Activation function: 'sigmoid' to produce an output between 0 and 1 (probability)
model_nn.add(Dense(units=1, activation='sigmoid'))

# Defines the Adam optimizer with a learning rate of 0.001
optimizer = Adam(learning_rate=0.001)

# Compiles the model specifying:
# - Loss function: 'binary_crossentropy', appropriate for binary classification
# - Optimizer: Adam
# - Metric: accuracy
model_nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Trains the neural network:
# - epochs: number of iterations over the dataset
# - batch_size: number of samples per weight update
# - validation_data: validation set to monitor performance during training
model_nn.fit(X_train_selected, y_train, epochs=200, batch_size=32, validation_data=(X_test_selected, y_test), verbose=1)

# Makes predictions on the test set using the neural network model
y_hat_nn = model_nn.predict(X_test_selected)

# Converts predictions (probabilities) into classes:
# If probability is less than 0.5, assign 0; otherwise, assign 1
y_hat_nn = [0 if val < 0.5 else 1 for val in y_hat_nn]

# Calculates and prints the neural network model's accuracy
print(f"Neural Network model accuracy: {accuracy_score(y_test, y_hat_nn):.4f}")

Epoch 1/200
177/177 [==============================] - 1s 2ms/step - loss: 0.4825 - accuracy: 0.7677 - val_loss: 0.4235 - val_accuracy: 0.7949
Epoch 2/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4402 - accuracy: 0.7918 - val_loss: 0.4204 - val_accuracy: 0.7949
Epoch 3/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4370 - accuracy: 0.7920 - val_loss: 0.4201 - val_accuracy: 0.7999
Epoch 4/200
177/177 [==============================] - 0s 3ms/step - loss: 0.4334 - accuracy: 0.7929 - val_loss: 0.4161 - val_accuracy: 0.8027
Epoch 5/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4349 - accuracy: 0.7936 - val_loss: 0.4171 - val_accuracy: 0.8055
Epoch 6/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4302 - accuracy: 0.7970 - val_loss: 0.4169 - val_accuracy: 0.8041
Epoch 7/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4342 - accuracy: 0.7929 - val_loss: 0.4148 - val_accuracy: 0.8055

Epoch 58/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4172 - accuracy: 0.8041 - val_loss: 0.4175 - val_accuracy: 0.7999
Epoch 59/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4162 - accuracy: 0.8039 - val_loss: 0.4130 - val_accuracy: 0.8006
Epoch 60/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4143 - accuracy: 0.8055 - val_loss: 0.4125 - val_accuracy: 0.7956
Epoch 61/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4139 - accuracy: 0.8089 - val_loss: 0.4139 - val_accuracy: 0.7991
Epoch 62/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4132 - accuracy: 0.8032 - val_loss: 0.4167 - val_accuracy: 0.7999
Epoch 63/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4125 - accuracy: 0.8066 - val_loss: 0.4143 - val_accuracy: 0.8034
Epoch 64/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4156 - accuracy: 0.8082 - val_loss: 0.4133 - val_accuracy:

Epoch 115/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4099 - accuracy: 0.8057 - val_loss: 0.4162 - val_accuracy: 0.7984
Epoch 116/200
177/177 [==============================] - 0s 3ms/step - loss: 0.4094 - accuracy: 0.8053 - val_loss: 0.4164 - val_accuracy: 0.7956
Epoch 117/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4085 - accuracy: 0.8092 - val_loss: 0.4160 - val_accuracy: 0.7935
Epoch 118/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4097 - accuracy: 0.8071 - val_loss: 0.4166 - val_accuracy: 0.7977
Epoch 119/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4107 - accuracy: 0.8062 - val_loss: 0.4162 - val_accuracy: 0.7970
Epoch 120/200
177/177 [==============================] - 1s 3ms/step - loss: 0.4095 - accuracy: 0.8087 - val_loss: 0.4171 - val_accuracy: 0.7928
Epoch 121/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4099 - accuracy: 0.8087 - val_loss: 0.4188 - val_ac

177/177 [==============================] - 0s 2ms/step - loss: 0.4063 - accuracy: 0.8073 - val_loss: 0.4234 - val_accuracy: 0.7963
Epoch 172/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4046 - accuracy: 0.8075 - val_loss: 0.4240 - val_accuracy: 0.7999
Epoch 173/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4052 - accuracy: 0.8080 - val_loss: 0.4193 - val_accuracy: 0.7963
Epoch 174/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4060 - accuracy: 0.8076 - val_loss: 0.4199 - val_accuracy: 0.7970
Epoch 175/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4068 - accuracy: 0.8108 - val_loss: 0.4223 - val_accuracy: 0.7977
Epoch 176/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4090 - accuracy: 0.8073 - val_loss: 0.4203 - val_accuracy: 0.7963
Epoch 177/200
177/177 [==============================] - 0s 2ms/step - loss: 0.4045 - accuracy: 0.8092 - val_loss: 0.4212 - val_accuracy: 0.7963